# Perform FMQA with Covalent and Fixstars Amplify

This sample code uses Covalent and Fixstars Amplify to perform FMQA.

The code is a modified version of [Amplify Examples](https://github.com/fixstars/amplify-examples/blob/main/notebooks/ja/examples/fmqa_0_algebra.ipynb). Amplify Examples is open source software under the [MIT License](https://github.com/fixstars/amplify-examples/blob/main/LICENSE).

## Environment Setup

### Create SSH Tunnel

To connect to the ABCI system via SSH, create an SSH tunnel or `~/.ssh/config` to be able to login using ProxyJump with reference to https://docs.abci.ai/ja/getting-started/#general-method.
The sample applications assume that you have created an SSH tunnel using port 10022 on localhost by the following command:

```console
ssh -i /path/identity_file -L 10022:es:22 -l username as.abci.ai
```

### Steps to create a virtual environment on the ABCI system

Login to the ABCI system and create a virtual environment with the necessary packages installed to perform the tasks in the sample applications.
Please refer to [Dependent Packages](#dependencies) for the required packages.
For more information on creating a virtual environment on the ABCI system, please refer to https://docs.abci.ai/ja/python/.

Below are the steps to create a virtual environment on the ABCI system named `amplify_env`.

1. Login to the ABCI system using your own account.
2. Run `module load python/3.10/3.10.10` to make python available.
3. Create a virtual environment by running `python3 -m venv amplify_env`. The `amplify_env` directory is also created at this time.

<a id="dependencies"></a>
### Dependent Packages

Install the packages required to run the application in your local machine environment and in the virtual environment on the ABCI system.  
Each sample application requires the packages listed below.  
Bolded packages are required for both the local machine and the virtual environment on the ABCI system, and the rest are required only for the local machine.

The sample application uses covalent-gridengine-plugin.  
Please refer to the README of covalent-gridengine-plugin for information on how to install and use covalent-gridengine-plugin.

- **covalent**
- **numpy**
- **torch**
- **scikit-learn**
- amplify<=0.12.1
- matplotlib
- covalent-gridengine-plugin

In [1]:
import covalent as ct

Create a GridEngineExecutor object to execute tasks on the ABCI system.

In [2]:
abci_executor = ct.executor.GridEngineExecutor(
    username="username",      # Enter your ABCI username.
    address="localhost",
    port=10022,
    ssh_key_file="~/.ssh/id_rsa",  # Enter the path to your ssh key file.
    remote_workdir="$HOME/amplify_env",
    poll_freq=30,
    cleanup=True,
    embedded_qsub_args={
        "l": ["rt_F=1", "h_rt=1:00:00"],
    },  # qsub options to be embedded in the script
    qsub_args={
        "g": "groupname",    # Enter your ABCI groupname.
    },  # qsub options to be given when it is run on the command line
    prerun_commands=[
        "source /etc/profile.d/modules.sh",
        "module load python/3.10/3.10.10",
        "module load cuda/11.8/11.8.0",
        "module load cudnn/8.8/8.8.1",
        "source ~/amplify_env/bin/activate",
    ],
    postrun_commands=[],
    bashrc_path="~/.bashrc",
    log_stdout="stdout.log",
    log_stderr="stderr.log",
)

Prepare the necessary functions to run FMQA.

The following is an excerpt of code from [Amplify Examples](https://github.com/fixstars/amplify-examples/blob/main/notebooks/ja/examples/fmqa_0_algebra.ipynb) and modified to run using Covalent.

In [3]:
import torch
import numpy as np

import torch.nn as nn


class TorchFM(nn.Module):
    def __init__(self, d: int, k: int) -> None:
        super().__init__()
        self.V = nn.Parameter(torch.randn(d, k), requires_grad=True)
        self.lin = nn.Linear(d, 1)  # all coupled network

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out_1 = torch.matmul(x, self.V).pow(2).sum(1, keepdim=True)
        out_2 = torch.matmul(x.pow(2), self.V.pow(2)).sum(1, keepdim=True)
        out_inter = 0.5 * (out_1 - out_2)
        out_lin = self.lin(x)
        out = out_inter + out_lin
        return out

In [6]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

import copy

# This task is executed on the local machine.
# A function that machine learns FM from I/O data
@ct.electron(executor=abci_executor)
def train(
    X: np.ndarray,
    y: np.ndarray,
    model_class: type[torch.nn.Module] | None = None,
    model_params: dict | None = None,
    batch_size: int = 1024,
    epochs: int = 3000,
    criterion: torch.nn.Module | None = None,
    optimizer_class: type[torch.optim.Optimizer] | None = None,
    opt_params: dict | None = None,
    lr_sche_class: type[torch.optim.lr_scheduler.LRScheduler] | None = None,
    lr_sche_params: dict | None = None,
) -> torch.nn.Module:
    X_tensor, y_tensor = (
        torch.from_numpy(X).float(),
        torch.from_numpy(y).float(),
    )
    indices = np.array(range(X.shape[0]))
    indices_train, indices_valid = train_test_split(
        indices, test_size=0.2, random_state=42
    )

    train_set = TensorDataset(X_tensor[indices_train], y_tensor[indices_train])
    valid_set = TensorDataset(X_tensor[indices_valid], y_tensor[indices_valid])
    loaders = {
        "train": DataLoader(train_set, batch_size=batch_size, shuffle=True),
        "valid": DataLoader(valid_set, batch_size=batch_size, shuffle=False),
    }

    model = model_class(**model_params)
    best_model_wts = copy.deepcopy(model.state_dict())
    optimizer = optimizer_class(model.parameters(), **opt_params)
    if lr_sche_class is not None:
        scheduler = lr_sche_class(optimizer, **lr_sche_params)
    best_score = 1e18
    for epoch in range(epochs):
        losses = {"train": 0.0, "valid": 0.0}

        for phase in ["train", "valid"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            for batch_x, batch_y in loaders[phase]:
                optimizer.zero_grad()
                out = model(batch_x).T[0]
                loss = criterion(out, batch_y)
                losses[phase] += loss.item() * batch_x.size(0)

                with torch.set_grad_enabled(phase == "train"):
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

            losses[phase] /= len(loaders[phase].dataset)

        with torch.no_grad():
            model.eval()
            if best_score > losses["valid"]:
                best_model_wts = copy.deepcopy(model.state_dict())
                best_score = losses["valid"]
        if lr_sche_class is not None:
            scheduler.step()

    with torch.no_grad():
        model.load_state_dict(best_model_wts)
        model.eval()
    return model

In [7]:
from collections.abc import Callable

# This task is executed on ABCI.
# A function that evaluates the objective function for input values and creates N0 input-output pairs (initial training data).
@ct.electron(executor=abci_executor)
def gen_training_data(D: int, N0: int, true_func: Callable) -> tuple[np.ndarray, np.ndarray]:
    assert N0 < 2**D

    # Generate N0 input values using random numbers.
    X = np.random.randint(0, 2, size=(N0, D))

    # Exclude duplicate input values from the input values.
    # And add the excluded input values using random numbers.
    X = np.unique(X, axis=0)
    while X.shape[0] != N0:
        X = np.vstack((X, np.random.randint(0, 2, size=(N0 - X.shape[0], D))))
        X = np.unique(X, axis=0)
    y = np.zeros(N0)

    # Get the output value corresponding to the N0 input values by evaluating the objective function.
    for i in range(N0):
        if i % 10 == 0:
            print(f"Generating {i}-th training data set.")
        y[i] = true_func(X[i])
    return X, y

Note that the sample code below is implemented with amplify v0.12 and is not guaranteed to work with amplify v1.0.0 or later.

In [8]:
import amplify

# Defining FM as QUBO from FM parameters.
# As with the previously defined TorchFM class, the formula is written as g(x).
def FM_as_QUBO(x: np.ndarray, w0: np.ndarray, w: np.ndarray, v: np.ndarray, k: int) -> amplify.BinaryPoly:
    lin = w0 + (x.T @ w)
    D = w.shape[0]
    out_1 = amplify.sum_poly(k, lambda i: amplify.sum_poly(D, lambda j: x[j] * v[j, i]) ** 2)

    # Note that x[j] is equivalent x[j] * x[j] because x[j] is a binary variable.
    out_2 = amplify.sum_poly(
        k, lambda i: amplify.sum_poly(D, lambda j: x[j] * v[j, i] * v[j, i])
    )
    return lin + (out_1 - out_2) / 2

# This task is executed on the local machine.
# A function that performs a solution using a Ising machine.
@ct.electron
def step(model: amplify.BinaryPoly, D: int, k: int) -> np.ndarray:
    client = amplify.client.FixstarsClient()
    client.parameters.timeout = 1000
    client.token = "xxxxxxxxxxxxxxx"  # Enter your token of Amplify AE.
    solver = amplify.Solver(client)

    v, w, w0 = list(model.parameters())
    v = v.detach().numpy()
    w = w.detach().numpy()[0]
    w0 = w0.detach().numpy()[0]

    gen = amplify.BinarySymbolGenerator()
    q = gen.array(D)
    cost = FM_as_QUBO(q, w0, w, v, k)  # Defining FM as QUBO from FM parameters.
    result = solver.solve(cost)  # Solve QUBO using Ising machine.
    if len(result.solutions) == 0:
        raise RuntimeError("No solution was found.")
    values = result.solutions[0].values
    q_values = q.decode(values)
    return q_values


import matplotlib.pyplot as plt

# A function that plots the history of objective function evaluation values for the initial training data and the i-th FMQA cycle.
def plot_history(y: np.ndarray, N: int, N0: int) -> plt.Figure:
    assert y is not None
    fig = plt.figure(figsize=(6, 4))
    plt.plot(
        [i for i in range(N0)],
        y[: N0],
        marker="o",
        linestyle="-",
        color="b",
    )  # Objective function evaluation value for the initial training data (random process)

    plt.plot(
        [i for i in range(N0, N)],
        y[N0 :],
        marker="o",
        linestyle="-",
        color="r",
    )  # Objective function evaluation value for the i-th FMQA cycle.
    plt.xlabel("i-th evaluation of f(x)", fontsize=18)
    plt.ylabel("f(x)", fontsize=18)
    plt.tick_params(labelsize=18)
    plt.show()
    return fig

In [10]:
# Create a d-dimensional symmetric matrix with zero mean of the components.
def make_Q(d: int) -> np.ndarray:
    Q_true = np.random.rand(d, d)
    Q_true = (Q_true + Q_true.T) / 2
    Q_true = Q_true - np.mean(Q_true)
    return Q_true

def get_true_func(D: int) -> Callable:
    Q = make_Q(D)
    def true_func(x):
        return x @ Q @ x
    return true_func


In [11]:
# This task is executed on ABCI.
@ct.electron(executor=abci_executor)
def append_new_data(
    X: np.ndarray,
    y: np.ndarray,
    x_hat: np.ndarray,
    pred_x: np.ndarray,
    pred_y: np.ndarray,
    D: int,
    true_func: Callable,
) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    # If the same input value already exists in the training data as x_hat, the surrounding values are used as x_hat.
    is_identical = True
    while is_identical:
        is_identical = False
        for j in range(X.shape[0]):
            if np.all(x_hat == X[j, :]):
                change_id = np.random.randint(0, D, 1)
                x_hat[change_id.item()] = 1 - x_hat[change_id.item()]
                is_identical = True
                break
    y_hat = true_func(x_hat)

    # Add the input-output pair [x_hat, y_hat] at the optimal point to the training data.
    X = np.vstack((X, x_hat))
    y = np.append(y, y_hat)

    # If the objective function evaluation value is updated to the minimum value, copy the input-output pair to [pred_x, pred_y].
    if pred_y > y_hat:
        pred_y = y_hat
        pred_x = x_hat
        print(f"variable updated, {pred_y=}")
    else:
        print("")
    return pred_x, pred_y, X, y

Combine the above 4 `electron` to create a `lattice` workflow.

In [12]:
@ct.lattice
def workflow(D: int, N: int, N0: int, k: int) -> tuple[np.ndarray, np.ndarray]:
    true_func = get_true_func(D)
    X, y = gen_training_data(D, N0, true_func)
    print(f"Starting FMQA cycles...")
    pred_x = X[0]
    pred_y = 1e18
    for i in range(N - N0):
        print(f"FMQA Cycle #{i} ", end="")
        model = train(X, y,
            model_class=TorchFM,
            model_params={"d": D, "k": k},
            batch_size=8,
            epochs=2000,
            criterion=nn.MSELoss(),
            optimizer_class=torch.optim.AdamW,
            opt_params={"lr": 1},)
        x_hat = step(model, D, k)
        pred_x, pred_y, X, y = append_new_data(X, y, x_hat, pred_x, pred_y, D, true_func)

        # If all inputs have been fully searched, exit the for loop.
        #if len(y) >= 2**D:
            #print(f"Fully searched. Terminating FMQA cycles.")
            #return None

    return pred_x, y


In [ ]:
# Input dimension (problem size)
D = 100
N = 70  # Number of times the function can be evaluated
N0 = 60  # Number of initial training data samples
k = 10  # Dimension of the vector in FM (hyper parameter)

Next, start the covalent server with the following command

```console
covalent start
````

With the default configuration, you can view the Covalent GUI by accessing http://localhost:48008/ from your browser.

Finally, execute the workflow. First, dispatch the workflow.

In [ ]:
dispatch_id = ct.dispatch(workflow)(D, N, N0, k)

In [ ]:
print(dispatch_id)

In [ ]:
result = ct.get_result(dispatch_id, wait=True)
print(result)

Plot the history of the objective function evaluation values.

In [ ]:
pred_x, y = result.result

fig = plot_history(y, N, N0)